# Predicting Salaries for MLB Players
### Kevin Payton
### PSTAT 131 Fall 24

---
## MLB Information

---
## Loading our Data and Libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

In [7]:
master = pd.read_csv('Master.csv')


---
## Exploring, Merging, and Tidying our Data

First I will import all my libraries:

I will first import my data I have decided on so far. Everything comes from Kaggle at the following link: https://www.kaggle.com/datasets/open-source-sports/baseball-databank?resource=download. So far I have decided on using the Master.csv, Salaries.csv, and Batting.csv files which all have data from 1871 through 2015, but I will only be working with data from year 2000 to 2015. The Master.csv file contains personal information of all the players in the dataset, but I will only be using this to gather the names of players to make importing any future data easier since currently my datasets rely on player IDs. The Salaries.csv file contains team and league information, as well as the salary players have for any given year. Finally, my Batting.csv file contains various batting statistics for players.

In [8]:
master['nameFull'] = master['nameFirst'] + ' ' + master['nameLast']
names = master[['playerID', 'nameFull']]

salaries = pd.read_csv('Salaries.csv')
salaries_post_2000 = salaries[salaries['yearID'] >= 2000]
print(salaries_post_2000.shape)

merged_data = pd.merge(salaries_post_2000, names, on='playerID', how='inner')
print(merged_data.shape)

merged_data_left = pd.merge(salaries_post_2000, names, on='playerID', how='left')
print(merged_data_left.shape)

(13312, 5)
(13304, 6)
(13312, 6)


After importing my Master.csv file, I first decided to make a full name column because the player's first and last names were separated. I then removed all columns except the playerID and nameFull to make the merging with salaries easier since all the other columns were unneeded. As you can see from the output above we are then left with 18846 rows and 2 columns, with the rows each representing players. We are left with so many because we currently still have values from every player that has played since 1871 through 2015, but all those prior to 2000 will be removed once we merge with our salaries dataset. 

I then imported my Salaries.csv file and filtered out all player data prior to 2000 to simplify things, leaving us with 13312 rows and 5 columns containing simple data about a player, the team they started the season and its league, as well as the year and their salary. This file does not contain information on if a player was traded during the year, because their salary stays the same. This is an issue I might have later when dealing with the batting information, because there might be multiple rows for the same player during a given year.

I was then able to finally merge the two datasets together, and I did so with both a left and an inner join so I could see if we were missing any information on any player's names. As you can see above, we were missing 8 player's names from the names data frame so I will create a function to discover the index of these players, as well as their playerID so I can try and find some more information about them in our Master.csv file.

In [9]:
null_data = merged_data_left['nameFull'].isnull()
missing_names = []

for ind in merged_data_left.index:
    if null_data.loc[ind] == True:
        missing_names.append(merged_data_left.loc[ind, 'playerID'])
    
print(missing_names)

['harriwi10', 'castiru02', 'rosajo01', 'arrueba01', 'furcara02', 'castiru02', 'rosajo01', 'harriwi10']


In [10]:
for playerid in missing_names:
    print(master.loc[master['playerID'] == playerid])

Empty DataFrame
Columns: [playerID, birthYear, birthMonth, birthDay, birthCountry, birthState, birthCity, deathYear, deathMonth, deathDay, deathCountry, deathState, deathCity, nameFirst, nameLast, nameGiven, weight, height, bats, throws, debut, finalGame, retroID, bbrefID, nameFull]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [playerID, birthYear, birthMonth, birthDay, birthCountry, birthState, birthCity, deathYear, deathMonth, deathDay, deathCountry, deathState, deathCity, nameFirst, nameLast, nameGiven, weight, height, bats, throws, debut, finalGame, retroID, bbrefID, nameFull]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [playerID, birthYear, birthMonth, birthDay, birthCountry, birthState, birthCity, deathYear, deathMonth, deathDay, deathCountry, deathState, deathCity, nameFirst, nameLast, nameGiven, weight, height, bats, throws, debut, finalGame, retroID, bbrefID, nameFull]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [playerID, birthYear, bi

Unfortunately, I was unable to find these names simply from going through my full Master.csv file, so I will have to do some extra research in case the ID was simply input wrong.

I was first able to discover that the player with the ID 'harriwi10' was Will Harris and was given the ID 'harriwi02' in both Master.csv and Batting.csv. The player with the ID 'castiru02' was Rusney Castillo with the ID 'castiru01'. The player with the ID 'rosajo01' was Jorge De La Rosa and also had data under the ID 'delarjo01'. The player with the ID 'arrueba01' was Erisbel Arruebarrena who also had data under the ID 'arrueer01'. Finally, the player with the ID 'furcara02' was Rafael Furcal who also had data under 'furcara01'. Before I do any work with my data, I will go through and fix these inconsistencies, but for now I have outlined what needs to be done.

In [20]:
salaries_post_2000.loc[''] = 'harriwi02'
print(salaries_post_2000[salaries_post_2000['playerID'] == 'harriwi02'])

          yearID     teamID       lgID   playerID     salary
23965  harriwi02  harriwi02  harriwi02  harriwi02  harriwi02
25044  harriwi02  harriwi02  harriwi02  harriwi02  harriwi02


C:\Users\kpayt\AppData\Local\Temp\ipykernel_34320\2041775303.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salaries_post_2000[salaries_post_2000['playerID'] == 'harriwi10'] = 'harriwi02'
C:\Users\kpayt\AppData\Local\Temp\ipykernel_34320\2041775303.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'harriwi02' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  salaries_post_2000[salaries_post_2000['playerID'] == 'harriwi10'] = 'harriwi02'
C:\Users\kpayt\AppData\Local\Temp\ipykernel_34320\2041775303.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas

In [13]:
batting_stats = pd.read_csv('Batting.csv')
batting_stats = batting_stats[batting_stats['yearID'] >= 2000]
print(batting_stats.shape)

new_merged = pd.merge(batting_stats, merged_data, on=['playerID', 'yearID'], how='inner')

final_data = new_merged[['playerID', 'nameFull', 'yearID', 'salary', 'teamID_x', 'stint',
                        'lgID_x', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS',
                        'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP']]

(22084, 22)
playerID    0
nameFull    0
yearID      0
salary      0
teamID_x    0
stint       0
lgID_x      0
G           0
AB          0
R           0
H           0
2B          0
3B          0
HR          0
RBI         0
SB          0
CS          0
BB          0
SO          0
IBB         0
HBP         0
SH          0
SF          0
GIDP        0
dtype: int64
